In [1]:
from pathlib import Path
from topagnps2cche1d import Watershed

import pandas as pd

In [2]:
path_to_reaches = Path('../data_inputs/goodwin_creek_1m/AnnAGNPS_Reach_Data_Section.csv')
path_to_agflow_reaches = Path('../data_inputs/goodwin_creek_1m/AgFlow_Reach_Data.csv')

In [3]:
watershed = Watershed()

In [4]:
watershed.import_topagnps_reaches_network(path_to_reaches)

In [5]:
watershed.assign_strahler_number_to_reaches()

In [6]:
watershed.ignore_reaches_with_strahler_leq(5)
watershed.update_graph()

In [7]:
reaches = watershed.reaches
for k, v in reaches.items():
    if not v.ignore:
        print(v)
    # break

-----------------------------
Reach           : 1
Receiving Reach : OUTLET
Slope           : 0.00283
Strahler Number : 6
Ignore?         : False
-----------------------------
Reach           : 2
Receiving Reach : 1
Slope           : 0.00283
Strahler Number : 6
Ignore?         : False
-----------------------------
Reach           : 3
Receiving Reach : 2
Slope           : 0.00134
Strahler Number : 6
Ignore?         : False
-----------------------------
Reach           : 4
Receiving Reach : 3
Slope           : 0.00097
Strahler Number : 6
Ignore?         : False
-----------------------------
Reach           : 5
Receiving Reach : 4
Slope           : 0.00172
Strahler Number : 6
Ignore?         : False
-----------------------------
Reach           : 33
Receiving Reach : 5
Slope           : 0.00253
Strahler Number : 6
Ignore?         : False
-----------------------------
Reach           : 34
Receiving Reach : 33
Slope           : 0.00175
Strahler Number : 6
Ignore?         : False
------------

In [8]:
df_agflow = pd.read_csv(path_to_agflow_reaches, index_col=False)

In [9]:
df_agflow.head()

,Reach_ID,Upstream_End_Row,Upstream_End_Column,Downstream_End_Row,Downstream_End_Column,Receiving_Reach,Drainage_Area_[ha],Average_Elevation_[m],Reach_Length_[m],Distance_Upstream_End_to_Outlet_[m],Distance_Downstream_End_to_Outlet_[m],Reach_Slope_[m/m],Contributing_Cell_ID_Source,Contributing_Cell_ID_Right,Contributing_Cell_ID_Left
0,1,6048,578,6048,578,1,2135.57,67.45,0.00,0.00,0.00,0.00283,0,0,0
1,2,5978,761,6048,578,1,2135.57,67.45,247.14,247.14,0.00,0.00283,0,22,23
2,3,5947,822,5978,762,2,2128.12,67.82,75.01,322.15,247.14,0.00134,0,32,33
3,4,5861,859,5946,823,3,2103.89,67.94,103.91,426.06,322.15,0.00097,0,42,43
4,5,5556,1051,5860,859,4,2101.50,68.30,406.73,832.79,426.06,0.00172,0,52,53


In [10]:
df = pd.read_csv(path_to_reaches, index_col=False)

In [12]:
df.head()

,Reach_ID,Receiving_Reach,Vegetation_Code,Elevation,Slope,Mannings_n,Infiltration_Rate,Hydraulic_Geom_ID,Length,Top_Width,...,Sand_Scour_Code,Small_Agg_Scour_Code,Large_Agg_Scour_Code,Valley_Clay_Scour_Code,Valley_Silt_Scour_Code,Valley_Sand_Scour_Code,Valley_Small_Agg_Scour_Code,Valley_Large_Agg_Scour_Code,Delivery_Ratio,Input_Units_Code
0,1,OUTLET,NaN,67.45,0.00283,NaN,NaN,NaN,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2,1,NaN,67.45,0.00283,NaN,NaN,NaN,247.14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,3,2,NaN,67.82,0.00134,NaN,NaN,NaN,75.01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,4,3,NaN,67.94,0.00097,NaN,NaN,NaN,103.91,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,5,4,NaN,68.30,0.00172,NaN,NaN,NaN,406.73,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
